In [18]:
import numpy as np
import networkx as nx
import gzip as gz

## Create Psis lists

In [6]:
psi_model = dict()
i=0
for line in open("Psi_model.txt"):
    line = line.split()
    psi_model[int(line[0])] = float(line[1])
    i+=1
print("Nb entrées pour model : ", i)

Nb entrées pour model :  5804715


In [7]:
psi_emul = dict()
i = 0
for line in open("Psi_emul.txt"):
    line = line.split()
    psi_emul[int(line[0])] = float(line[1])
    i += 1
print("Nb entrées pour emul : ", i)

Nb entrées pour emul :  3754043


Are users with psi>0 the same in the 2 lists ?

In [4]:
set([u for u in psi_model if psi_model[u]>0]) == set([u for u in psi_emul if psi_emul[u]>0])

True

## Create graph and activity

Graph

In [7]:
G = nx.DiGraph()

# get data
tweets = open("../wcano_tronc.txt", 'r')

# author dict
Author = dict()
for tweet in tweets:
    tweet = tweet.split()
    twid, uid = int(tweet[0]), int(tweet[2])
    Author[twid] = uid
        
# create edges
tweets.seek(0)
for tweet in tweets:
    tweet = tweet.split()
    twid, uid, rtid = int(tweet[0]), int(tweet[2]), int(tweet[-1])
    if rtid != -1:
        if rtid in Author.keys():
            G.add_edge(Author[rtid], uid)

# close
tweets.close()

Activity

In [19]:
users = set()
count = {'tweets':dict(), 'retweets':dict(), 'retweeted':dict()}
tweets = gz.open("../tweets.tronques.sorted22n.txt.gz", 'r')
    
# parcourt tweets
for i,tweet in enumerate(tweets):
    tweet = tweet.split()
    uid, rtid = int(tweet[2]), int(tweet[-1])

    # si user non connu on crée des nouvelles entrées de dictionnaire
    if uid not in users:
        users.add(uid)
        count['tweets'][uid] = 0
        count['retweets'][uid] = 0
        count['retweeted'][uid] = 0

    # si tweet original update nb_tweets
    if rtid == -1:
        count['tweets'][uid] += 1
        
    # si retweet
    else:
        if rtid in Author.keys():
            count['retweets'][uid] += 1
            rtu = Author[rtid]
            if rtu in users:
                count['retweeted'][rtu] += 1
            else:
                count['tweets'][rtu], count['retweets'][rtu], count['retweeted'][rtu] = 0, 0, 1
        else:
            count['tweets'][uid] += 1

    # on enregistre le ts du 1er tweet et celui du dernier tweet (update à chaque étape car tweets pas forcément pas ordre chrono)
    ts = int(tweet[1])
    if i==0:
        first_ts = ts
        last_ts = ts
    elif ts < first_ts:
        first_ts = ts
    elif ts > last_ts:
        last_ts = ts
            

total_time = last_ts - first_ts
tweets.close()

NameError: name 'Author' is not defined

Look at graph stats and activity for users that are not in common between the 2 lists

In [16]:
model_not_emul = {'tweets':list(), 'retweets':list(), 'retweeted':list(), 'outdeg':list(), 'indeg':list()}
emul_not_model = {'tweets':list(), 'retweets':list(), 'retweeted':list(), 'outdeg':list(), 'indeg':list()}

# model not emul
for u in psi_model:
    if u not in psi_emul:
        model_not_emul['tweets'].append(count['tweets'][u])
        model_not_emul['retweets'].append(count['retweets'][u])
        model_not_emul['retweeted'].append(count['retweeted'][u])
        model_not_emul['outdeg'].append(G.out_degree(u))
        model_not_emul['indeg'].append(G.in_degree(u))
    
# emul not model
for u in psi_emul:
    if u not in psi_model:
        emul_not_model['tweets'].append(count['tweets'][u])
        emul_not_model['retweets'].append(count['retweets'][u])
        emul_not_model['retweeted'].append(count['retweeted'][u])
        emul_not_model['outdeg'].append(G.out_degree(u))
        emul_not_model['indeg'].append(G.in_degree(u))

In [20]:
print("Emul not model : ", emul_not_model)

Emul not model :  {'tweets': [], 'retweets': [], 'retweeted': [], 'outdeg': [], 'indeg': []}


All erros are entries of model that are not in emul. Stats on them :

In [23]:
for key in model_not_emul:
    print("{} --- Mean {} --- Med {} --- Min {} --- Max {}"
          .format(key, np.mean(model_not_emul[key]), np.median(model_not_emul[key]),
                 np.min(model_not_emul[key]), np.max(model_not_emul[key])))

tweets --- Mean 0.0 --- Med 0.0 --- Min 0 --- Max 0
retweets --- Mean 1.4520469387595871 --- Med 1.0 --- Min 1 --- Max 219
retweeted --- Mean 0.0 --- Med 0.0 --- Min 0 --- Max 0
outdeg --- Mean 0.0 --- Med 0.0 --- Min 0 --- Max 0
indeg --- Mean 1.2927323335960115 --- Med 1.0 --- Min 1 --- Max 78


We see that these problematic users are retweeters only that are never retweeted. Now we check if there are others retweeters-only-never-retweeted users that are in psi_model AND in psi_emul.

In [22]:
i = 0
for u in psi_model:
    if u not in model_not_emul:
        if count['tweets'][u]==0:
            if count['retweets'][u]>0:
                if count['retweeted'][u]==0:
                    if G.out_degree(u)==0:
                        if G.in_degree(u)==0:
                            i += 1
print("There are {} such users".format(i))

There are 0 such users


## Check q for problematic users

In [20]:
for u in psi_model:
    if u not in psi_emul:
        print(u, psi_model[u])
        
        for influence_of in open("Wcano.qWall.0.5804715"):
            for item in iter(influence_of.split()):
                if str(u)==item:
                    print(True)
                    break

1000001 0.0
True


KeyboardInterrupt: 

## Compare lists

In [4]:
myPsi = list()
for line in open("Wcano.Psi.0.5804715", "r"):
    line = line.split()
    myPsi.append((int(line[0]), float(line[1])))

In [5]:
myPsi = sorted(myPsi, key=lambda x:x[1], reverse=True)

In [6]:
myPsi[:10]

[(11604, 0.0354614),
 (92737, 0.0103674),
 (11374, 0.00788391),
 (131389, 0.00677407),
 (5597, 0.0062115),
 (17447, 0.00497693),
 (49933, 0.00495309),
 (51522, 0.00465352),
 (22694, 0.00429894),
 (119142, 0.0042815)]

In [7]:
oldPsi = list()
for line in open("Psi_model.txt", "r"):
    line = line.split()
    oldPsi.append((int(line[0]), float(line[1])))

In [8]:
oldPsi = sorted(oldPsi, key=lambda x:x[1], reverse=True)

In [13]:
len(myPsi)

5804715

In [15]:
for i,item in enumerate(oldPsi):
    monpsi = myPsi[i]
    if item != monpsi and item[1] != monpsi[1]:
        print(item, monpsi)